Problem Statement:

Given a function $f(x)$ on n-qubits $\ket{x}$, that has an n-bit secret code, $a$, such that $f(x) = a.x = (a0x0+a1x1+...) \bmod 2$.

Find the secret code, $a$.

# Imports

In [ ]:
import qckt
import qckt.backend as bknd
import random as rand

# The function $f(x)$

In [ ]:
## The function with the secret code
def get_fxop(inreg, outreg):
    ## fx has inreg register as inputs, and one qubit output
    inregsz = len(inreg)
    outregsz = len(outreg) # ASSERTION: outreg size is 1
    secret_code = rand.randint(0,2**inregsz-1)
    print(("Pssst... the secret code is {:0"+str(inregsz)+"b}").format(secret_code))

    fxckt = qckt.QCkt(inregsz+outregsz)
    for i in range(inregsz):
        if secret_code & (0x1<<i):
            fxckt.CX(inreg[inregsz-1-i],outreg[0])
    # fxckt.draw()
    return fxckt.to_opMatrix()

## Qubits assignment for the algorithm

In [ ]:
fxsize = 6
fxin = [i for i in reversed(range(fxsize))]
fxout = [fxsize]
nqubits = fxsize + 1

## Get the secret function as an operator

In [ ]:
print("Getting the secret function blackbox...")
fxop = get_fxop(fxin,fxout)
print("OK, FX is ready.")

# The Bernstein-Vazirani algorithm

In [ ]:
bv_ckt = qckt.QCkt(nqubits=nqubits, nclbits=nqubits)

# Step 0: Prepare the result bit |b> to |->
bv_ckt.X(fxout)
bv_ckt.H(fxout)

# Step 1: Apply H on all qbits of |x>
bv_ckt.H(fxin)

# Step 2: Now apply the secret function f()
bv_ckt.Border()
bv_ckt.custom_gate('fx', fxop)
bv_ckt.fx(*(fxout+fxin))
bv_ckt.Border()

# Step 3: Again apply H on all qbits of |x>
bv_ckt.H(fxin)

# Step 4: Measure all qbits of |x>
bv_ckt.M(fxin)

bv_ckt.draw()

# Run the algorithm, and readout the result

In [ ]:
job = qckt.Job(bv_ckt,qtrace=False, verbose=False)
bknd.DMQdeb().runjob(job)
resval = job.get_creg()[0].intvalue
print(f'Secret code: {resval & (2**fxsize - 1):0{fxsize}b}')
